<a href="https://colab.research.google.com/github/amitnits2016/Keras/blob/master/RetinaNet/Prepare_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/fizyr/keras-retinanet.git

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 5862, done.
remote: Total 5862 (delta 0), reused 0 (delta 0), pack-reused 5862
Receiving objects: 100% (5862/5862), 13.40 MiB | 11.54 MiB/s, done.
Resolving deltas: 100% (3948/3948), done.


In [2]:
cd keras-retinanet/

/content/keras-retinanet


In [3]:
ls

CONTRIBUTORS.md  keras_retinanet/  requirements.txt  snapshots/
examples/        LICENSE           setup.cfg         tests/
images/          README.md         setup.py


In [4]:
!python3 setup.py build install

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/keras_retinanet
copying keras_retinanet/__init__.py -> build/lib.linux-x86_64-3.6/keras_retinanet
copying keras_retinanet/initializers.py -> build/lib.linux-x86_64-3.6/keras_retinanet
copying keras_retinanet/losses.py -> build/lib.linux-x86_64-3.6/keras_retinanet
creating build/lib.linux-x86_64-3.6/tests
copying tests/__init__.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_losses.py -> build/lib.linux-x86_64-3.6/tests
creating build/lib.linux-x86_64-3.6/keras_retinanet/callbacks
copying keras_retinanet/callbacks/coco.py -> build/lib.linux-x86_64-3.6/keras_retinanet/callbacks
copying keras_retinanet/callbacks/__init__.py -> build/lib.linux-x86_64-3.6/keras_retinanet/callbacks
copying keras_retinanet/callbacks/common.py -> build/lib.linux-x86_64-3.6/keras_retinanet/callbacks
copying keras_retinanet/callbacks/eval.py -> build/lib.linux-x86_64-3.6/keras_retinanet

In [5]:
ls

build/           images/                    README.md         snapshots/
CONTRIBUTORS.md  keras_retinanet/           requirements.txt  tests/
dist/            keras_retinanet.egg-info/  setup.cfg
examples/        LICENSE                    setup.py


In [6]:
%load /content/drive/My Drive/RetinaNet/data_load.py

In [7]:
%pycat /content/drive/My Drive/RetinaNet/data_load.py

In [8]:
%%writefile data_load.py
import os
import sys
import math
import glob
import argparse
import xml.etree.ElementTree as ET

# file locations
train_data = '/content/drive/My Drive/RetinaNet/Data/Train/'
test_data = '/content/drive/My Drive/RetinaNet/Data/Test/'

# output
train_annotations = './annotations.csv'
val_annotations = './val_annotations.csv'
label_file = './classes.csv'


# CONVERT the XML annotations to CSV format
def convert_annotation(train_data, test_data, image_id, filename, classes, train=False):
    if train:
        in_file = open(train_data + '%s.xml' % image_id)
    else:
        in_file = open(test_data + '%s.xml' % image_id)
    out_file = open(filename, 'a')
    tree = ET.parse(in_file)
    root = tree.getroot()

    if root.iter('object') is not None:
        for obj in root.iter('object'):
            cls = obj.find('name').text
            if cls not in classes:
                continue
            cls_id = classes.index(cls)

            xmlbox = obj.find('bndbox')
            x1 = math.ceil(float(xmlbox.find('xmin').text))
            y1 = math.ceil(float(xmlbox.find('ymin').text))
            x2 = math.ceil(float(xmlbox.find('xmax').text))
            y2 = math.ceil(float(xmlbox.find('ymax').text))
            if x1 == x2 or y1 == y2:
                continue
            if train:
                out_file.write(
                    f'{train_data + image_id}.jpg,{x1},{y1},{x2},{y2},{cls}\n')
            else:
                out_file.write(
                    f'{test_data + image_id}.jpg,{x1},{y1},{x2},{y2},{cls}\n')
    else:
        if train:
            out_file.write(f'{train_data + image_id}.jpg,,,,,\n')
        else:
            out_file.write(f'{test_data + image_id}.jpg,,,,,\n')


def parse_args(args):
    """ Parse the arguments.
    """
    parser     = argparse.ArgumentParser(description='Simple training script for training a RetinaNet network.')
    # subparsers = parser.add_subparsers(help='Arguments for specific dataset types.', dest='dataset_type')
    # subparsers.required = True

    # group = parser.add_mutually_exclusive_group()
    parser.add_argument('--train', help='Input dir with train images', default=' ', dest='train')
    parser.add_argument('--test',  help='Input dir for test images', default='', dest='test')

    return parser.parse_args(args)


def main(args=None):
    # parse arguments

    if args is None:
        args = sys.argv[1:]
    args = parse_args(args)

    train_data = args.train
    test_data = args.test

    # filter out the train test jpg files
    train_image = glob.glob(train_data + '/*.jpg')

    test_image = glob.glob(test_data + '/*.jpg')

    # define the classes for the model
    labels = ['chair', 'table', 'vase']

    # create new annotation files for  Train/validation
    open(train_annotations, 'w')
    open(val_annotations, 'w')

    train_ids = [os.path.basename(i[:-4]) for i in train_image]
    for image_id in train_ids:
        convert_annotation(train_data, test_data, image_id, train_annotations, labels, train=True)

    val_ids = [os.path.basename(i[:-4]) for i in test_image]

    for image_id in val_ids:
        convert_annotation(train_data, test_data, image_id, val_annotations, labels)

    # creating classes file listing all classes
    with open(label_file, 'w') as f:
        for i, line in enumerate(labels):
            f.write('{},{}\n'.format(line, i))

    print('DONE')


if __name__ == '__main__':
    main()

Writing data_load.py


In [10]:
%rm /content/drive/'My Drive'/RetinaNet/data_load.py